# Nutribot 

In [36]:
from dotenv import load_dotenv
load_dotenv()

True

## Basic API Calls

In [37]:
from openai import OpenAI
import os
import json

# create client
client = OpenAI()

# send a prompt
response = client.chat.completions.create(
    model="gpt-4o-mini",   # pick a model
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello, can you explain what protein is in nutrition terms?"}
    ]
)

print(json.dumps(response.to_dict(), indent=2))

{
  "id": "chatcmpl-CIlTXE0MQ8bmiKafkxleCCtJVzxAH",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Certainly! In nutritional terms, protein is one of the three macronutrients essential for human health, alongside carbohydrates and fats. Proteins are large, complex molecules made up of smaller units called amino acids, which are linked together in chains. There are 20 different amino acids, and the specific combination and sequence of these amino acids determine the structure and function of each protein.\n\n### Functions of Protein:\n1. **Building and Repairing Tissues**: Proteins are crucial for the growth, repair, and maintenance of body tissues, including muscles, skin, and organs.\n  \n2. **Enzymes**: Many proteins function as enzymes, which are catalysts that speed up biochemical reactions in the body.\n\n3. **Hormones**: Some proteins serve as hormones that regulate various physiological process

In [38]:
from openai import OpenAI
client = OpenAI()

messages = [
    {"role": "system", "content": "You are a helpful nutrition assistant."}
]

def chat(user_input: str):
    # add user msg
    messages.append({"role": "user", "content": user_input})

    # call API
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )

    reply = response.choices[0].message.content
    print("Bot:", reply)

    # add reply to history
    messages.append({"role": "assistant", "content": reply})

In [39]:
chat("What is protein?")

Bot: Protein is a macronutrient that is essential for the growth, repair, and maintenance of tissues in the body. It is made up of smaller units called amino acids, which are linked together in various sequences to form different types of proteins. There are 20 different amino acids that combine in various ways to create the myriad of proteins needed for the body.

Proteins play a crucial role in many biological processes, including:

1. **Building and Repairing Tissues**: Proteins are necessary for the structure of muscles, skin, organs, and other tissues.
  
2. **Enzymes and Hormones**: Many enzymes that catalyze biochemical reactions and hormones that regulate physiological functions are proteins.

3. **Immune Function**: Antibodies, which help protect the body against infections, are made of proteins.

4. **Transport and Storage**: Proteins can transport molecules (like hemoglobin transporting oxygen in the blood) and store nutrients.

5. **Energy Source**: Though not the primary f

In [40]:
chat("How much protein is in ground beef?")

Bot: The protein content in ground beef can vary slightly depending on the fat content of the meat. Generally, here are approximate protein values for different types of ground beef (cooked):

- **Lean Ground Beef (90% lean, 10% fat)**: About 22-24 grams of protein per 100 grams (approximately 3.5 ounces).
- **Regular Ground Beef (80% lean, 20% fat)**: About 25-27 grams of protein per 100 grams (approximately 3.5 ounces).
  
Keep in mind that these values can fluctuate based on cooking methods and specific brands or products. For the most accurate nutrition information, it's best to check the packaging or consult a specific nutrient database.


In [ ]:
messages # Past messages are maintained until "messages" is reset

[{'role': 'system', 'content': 'You are a helpful nutrition assistant.'},
 {'role': 'user', 'content': 'What is protein?'},
 {'role': 'assistant',
  'content': 'Protein is a macronutrient that is essential for the growth, repair, and maintenance of tissues in the body. It is made up of smaller units called amino acids, which are linked together in various sequences to form different types of proteins. There are 20 different amino acids that combine in various ways to create the myriad of proteins needed for the body.\n\nProteins play a crucial role in many biological processes, including:\n\n1. **Building and Repairing Tissues**: Proteins are necessary for the structure of muscles, skin, organs, and other tissues.\n  \n2. **Enzymes and Hormones**: Many enzymes that catalyze biochemical reactions and hormones that regulate physiological functions are proteins.\n\n3. **Immune Function**: Antibodies, which help protect the body against infections, are made of proteins.\n\n4. **Transport a

### Adding in a Tool

In [51]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_nutrition",
            "description": "Fetch nutrition data from local database.",
            "parameters": {
                "type": "object",
                "properties": {
                    "food": {"type": "string", "description": "The food to look up"}
                },
                "required": ["food"]
            }
        }
    }
]

def get_nutrition(food: str) -> dict:
    db = {
        "apple": {"calories": 95, "protein_g": 0.5, "carbs_g": 25, "fat_g": 0.3},
        "banana": {"calories": 105, "protein_g": 1.3, "carbs_g": 27, "fat_g": 0.3},
        "egg": {"calories": 78, "protein_g": 6.0, "carbs_g": 0.6, "fat_g": 5.3}
    }
    return db.get(food.lower(), {"calories": 0, "protein_g": 0, "carbs_g": 0, "fat_g": 0})


tool_map = {
    "get_nutrition": get_nutrition
}

messages = [
    {"role": "system", "content": "You are a helpful nutrition assistant."},
    {"role": "user", "content": "How many calories are in an apple?"}
]

resp = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools
)

msg = resp.choices[0].message
print("=== Model reply ===")
print(json.dumps(msg.to_dict(), indent=2))

if msg.tool_calls:
    tool_call = msg.tool_calls[0]
    tool_selection = tool_call.function.name
    args = json.loads(tool_call.function.arguments)
    result = tool_map[tool_selection](**args) 
    print("=== Python ran tool ===")
    print(json.dumps(result, indent=2))

    messages.append(msg)
    messages.append({"role":"tool","tool_call_id":tool_call.id,"content":json.dumps(result)})

    resp2 = client.chat.completions.create(
        model="gpt-4o-mini", messages=messages)

    final_reply = resp2.choices[0].message.content

    print("=== Final Bot Reply ===")
    print(final_reply)

=== Model reply ===
{
  "content": null,
  "refusal": null,
  "role": "assistant",
  "annotations": [],
  "tool_calls": [
    {
      "id": "call_xFGEl1k7VPxtbGTLuki3iJmc",
      "function": {
        "arguments": "{\"food\":\"apple\"}",
        "name": "get_nutrition"
      },
      "type": "function"
    }
  ]
}
=== Python ran tool ===
{
  "calories": 95,
  "protein_g": 0.5,
  "carbs_g": 25,
  "fat_g": 0.3
}
=== Final Bot Reply ===
An average medium-sized apple contains about 95 calories.


### Enforce Output Schema with `parse` and `pydantic`

In [ ]:
from openai import chat
from pydantic import BaseModel
import json


tools = [
    {
        "type": "function",
        "function": {
            "name": "get_nutrition",
            "description": "Fetch nutrition data from local database.",
            "parameters": {
                "type": "object",
                "properties": {
                    "food": {"type": "string", "description": "The food to look up"}
                },
                "required": ["food"]
            }
        }
    }
]

def get_nutrition(food: str) -> dict:
    db = {
        "apple": {"calories": 95, "protein_g": 0.5, "carbs_g": 25, "fat_g": 0.3},
        "banana": {"calories": 105, "protein_g": 1.3, "carbs_g": 27, "fat_g": 0.3},
        "egg": {"calories": 78, "protein_g": 6.0, "carbs_g": 0.6, "fat_g": 5.3}
    }
    return db.get(food.lower(), {"calories": 0, "protein_g": 0, "carbs_g": 0, "fat_g": 0})


tool_map = {
    "get_nutrition": get_nutrition
}

class NutritionResponse(BaseModel):
    food: str
    calories: int
    protein_g: float
    carbs_g: float
    fat_g: float

resp = client.chat.completions.parse(
    model="gpt-4.1-mini", 
    messages=[
        {"role": "user", "content": "Give me nutrition facts for an apple."}
    ],
    response_format=NutritionResponse  #Pass in Pydantic model
)

parsed = resp.choices[0].message.parsed
print(json.dumps(parsed.model_dump(), indent=2))

{
  "food": "Apple, raw, with skin",
  "calories": 52,
  "protein_g": 0.3,
  "carbs_g": 14.0,
  "fat_g": 0.2
}


### Defining a Basic Bot Class

In [64]:
from openai import OpenAI
from pydantic import BaseModel
import json

client = OpenAI()

# Schema
class NutritionResponse(BaseModel):
    food: str
    calories: int
    protein_g: float
    carbs_g: float
    fat_g: float

class NutriBot:
    def __init__(self, model="gpt-4.1-mini"):
        self.client = client
        self.model = model
        self.messages = [
            {"role": "system", "content": "You are NutriBot, a helpful nutrition assistant."}
        ]

    def chat(self, user_input: str) -> NutritionResponse:
        self.messages.append({"role": "user", "content": user_input})

        resp = self.client.chat.completions.parse(
            model=self.model,
            messages=self.messages,
            response_format=NutritionResponse
        )

        parsed = resp.choices[0].message.parsed

        self.messages.append({
            "role": "assistant",
            "content": parsed.model_dump_json()
        })

        return parsed

    def history(self):
        return print(json.dumps(self.messages, indent=2))

In [65]:
bot = NutriBot()

reply1 = bot.chat("Give me nutrition facts for an apple.")
print(reply1.model_dump())

reply2 = bot.chat("What about a banana?")
print(reply2.model_dump())

bot.history()

{'food': 'Apple', 'calories': 95, 'protein_g': 0.5, 'carbs_g': 25.0, 'fat_g': 0.3}
{'food': 'Banana', 'calories': 105, 'protein_g': 1.3, 'carbs_g': 27.0, 'fat_g': 0.3}
[
  {
    "role": "system",
    "content": "You are NutriBot, a helpful nutrition assistant."
  },
  {
    "role": "user",
    "content": "Give me nutrition facts for an apple."
  },
  {
    "role": "assistant",
    "content": "{\"food\":\"Apple\",\"calories\":95,\"protein_g\":0.5,\"carbs_g\":25.0,\"fat_g\":0.3}"
  },
  {
    "role": "user",
    "content": "What about a banana?"
  },
  {
    "role": "assistant",
    "content": "{\"food\":\"Banana\",\"calories\":105,\"protein_g\":1.3,\"carbs_g\":27.0,\"fat_g\":0.3}"
  }
]


### Parse out meal sentance

In [67]:
from pydantic import BaseModel
from typing import List

class FoodItem(BaseModel):
    food: str
    quantity: float
    unit: str
    calories: int
    protein_g: float
    carbs_g: float
    fat_g: float

class MealResponse(BaseModel):
    items: List[FoodItem]


resp = client.chat.completions.parse(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": "You are NutriBot, a nutrition assistant. "
        "The user will describe a meal in natural language. Your job is to extract each food, "
        "quantity, and unit, and estimate its calories, protein_g, carbs_g, and fat_g. "
        "Always return JSON that matches the MealResponse schema."},
        {"role": "user", "content": "I ate 2 eggs and a banana."}
    ],
    response_format=MealResponse
)
parsed = resp.choices[0].message.parsed
totals = {
    "calories": sum(item.calories for item in parsed.items),
    "protein_g": sum(item.protein_g for item in parsed.items),
    "carbs_g": sum(item.carbs_g for item in parsed.items),
    "fat_g": sum(item.fat_g for item in parsed.items)
}
print(totals)
print(parsed.model_dump_json(indent=2))

{'calories': 245, 'protein_g': 13.3, 'carbs_g': 28.0, 'fat_g': 10.3}
{
  "items": [
    {
      "food": "egg",
      "quantity": 2.0,
      "unit": "large",
      "calories": 140,
      "protein_g": 12.0,
      "carbs_g": 1.0,
      "fat_g": 10.0
    },
    {
      "food": "banana",
      "quantity": 1.0,
      "unit": "medium",
      "calories": 105,
      "protein_g": 1.3,
      "carbs_g": 27.0,
      "fat_g": 0.3
    }
  ]
}
